In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [4]:
X=pd.read_csv('../proyecto_ML/X.csv')
y=pd.read_csv('../proyecto_ML/y.csv')

In [8]:
X=X.drop('Unnamed: 0',axis=1)
y=y.drop('Unnamed: 0',axis=1)

In [9]:
X.head()

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,lat,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
0,1200,2,2.0,1,1,1,0,0,0,43.5851,...,0,0,0,0,0,0,0,0,0,0
1,694,1,1.0,1,1,1,0,0,0,38.9137,...,0,0,0,0,0,0,0,0,0,0
2,900,2,2.0,0,0,1,0,0,0,36.7922,...,0,0,0,0,1,0,0,0,0,0
3,1469,3,2.0,1,1,1,0,0,0,33.5623,...,0,0,0,0,0,0,0,0,0,0
4,700,1,1.0,1,1,1,0,0,0,36.0595,...,0,1,0,0,0,0,0,0,0,0


In [11]:
y.head()

,price
0,1350
1,1115
2,1129
3,1580
4,995


In [12]:
len(X),len(y)

(344757, 344757)

In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [16]:
#Realizar el split de los dataset para el entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state = 42)

In [17]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((241329, 490), (103428, 490), (241329, 1), (103428, 1))

In [18]:
#Crear el modelo Random Forest
modelo = RandomForestRegressor(
            n_estimators = 100,
            max_depth    = 10,
            n_jobs       = -1,
            random_state = 42
         )

In [19]:
modelo.fit(X_train, y_train)

C:\Users\Asus\AppData\Local\Temp\ipykernel_20160\2554334259.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  modelo.fit(X_train, y_train)


RandomForestRegressor(max_depth=10, n_jobs=-1, random_state=42)

In [20]:
predicciones = modelo.predict(X = X_test)

rmse = mean_squared_error(
        y_true  = y_test,
        y_pred  = predicciones,
        squared = False
       )
print(f"El error (rmse) de test es: {rmse}")

El error (rmse) de test es: 8607996.67012967


Importar el dataset de testeo y realizar su respectivo tratamiento antes de la prediccion

In [23]:
dftest=pd.read_parquet('../proyecto_ML/test.parquet')
dftest.hea()

,id,url,region,region_url,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7037609789,https://annarbor.craigslist.org/apa/d/wixom-ho...,ann arbor,https://annarbor.craigslist.org,manufactured,1344,3,2.0,0,0,...,0,0,0,w/d in unit,off-street parking,https://images.craigslist.org/00M0M_iNczP1nzIL...,"OPEN HOUSE TODAY! APPLY THIS WEEK, PUT A HOLDI...",42.5333,-83.5763,mi
1,7032406876,https://vermont.craigslist.org/apa/d/randolph-...,vermont,https://vermont.craigslist.org,apartment,1050,2,1.0,0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00L0L_ecirmYBIzL...,"Think of it, you'll be first to get your mail....",43.9393,-72.5538,vt
2,7037022682,https://annarbor.craigslist.org/apa/d/ann-arbo...,ann arbor,https://annarbor.craigslist.org,apartment,1150,2,2.0,1,1,...,1,0,0,w/d in unit,carport,https://images.craigslist.org/00e0e_dPln2xjo9g...,One of Ann Arbor's most luxurious apartment co...,42.2492,-83.7712,mi
3,7048681802,https://fortcollins.craigslist.org/apa/d/fort-...,fort collins / north CO,https://fortcollins.craigslist.org,apartment,1280,2,2.5,1,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00L0L_jlektT5cSd...,"Specials! Move in before January 16th, 2020 an...",40.5501,-105.0350,co
4,7043597870,https://charlottesville.craigslist.org/apa/d/c...,charlottesville,https://charlottesville.craigslist.org,apartment,783,2,1.0,1,1,...,0,0,0,laundry on site,None,https://images.craigslist.org/00D0D_cXa4KbZ6ox...,Barracks West Apartments & Townhomes in Charlo...,38.0936,-78.5611,va


In [24]:
#Revisar informacion general del DataFrame
dftest.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38498 entries, 0 to 38497
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       38498 non-null  int64  
 1   url                      38498 non-null  object 
 2   region                   38498 non-null  object 
 3   region_url               38498 non-null  object 
 4   type                     38498 non-null  object 
 5   sqfeet                   38498 non-null  int64  
 6   beds                     38498 non-null  int64  
 7   baths                    38498 non-null  float64
 8   cats_allowed             38498 non-null  int64  
 9   dogs_allowed             38498 non-null  int64  
 10  smoking_allowed          38498 non-null  int64  
 11  wheelchair_access        38498 non-null  int64  
 12  electric_vehicle_charge  38498 non-null  int64  
 13  comes_furnished          38498 non-null  int64  
 14  laundry_options       

In [25]:
#Eliminar comlumnas que no son relevantes para el estudio
dftest=dftest.drop(['id','url','region_url','image_url','description'], axis=1)

In [26]:
#Revisar a mayor detalle los valores nulos
dftest.isna().sum()

region                         0
type                           0
sqfeet                         0
beds                           0
baths                          0
cats_allowed                   0
dogs_allowed                   0
smoking_allowed                0
wheelchair_access              0
electric_vehicle_charge        0
comes_furnished                0
laundry_options             7855
parking_options            14005
lat                          196
long                         196
state                          0
dtype: int64

In [27]:
#Eliminar valores nulos en la columna long y lat
dftest = dftest[dftest.long.isna() == False]
dftest = dftest[dftest.lat.isna() == False]
len(dftest)

38302

In [29]:
#Reemplazar los valores nulos de laundry_options y parking_options
#por valores que indican ausencia de esta caracteristica
dftest.loc[dftest.laundry_options.isna()==True,'laundry_options']='no laundry on site'
dftest.loc[dftest.parking_options.isna()==True,'parking_options']='no parking'
dftest.isna().sum()

region                     0
type                       0
sqfeet                     0
beds                       0
baths                      0
cats_allowed               0
dogs_allowed               0
smoking_allowed            0
wheelchair_access          0
electric_vehicle_charge    0
comes_furnished            0
laundry_options            0
parking_options            0
lat                        0
long                       0
state                      0
dtype: int64

In [31]:
#Procesamiento de las variables categóricas en el dataFrame generando variables dummies
dftest=pd.get_dummies(dftest)
dftest

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,lat,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
0,1344,3,2.0,0,0,1,0,0,0,42.5333,...,0,0,0,0,0,0,0,0,0,0
1,1050,2,1.0,0,0,1,0,0,0,43.9393,...,0,0,0,0,0,1,0,0,0,0
2,1150,2,2.0,1,1,1,1,0,0,42.2492,...,0,0,0,0,0,0,0,0,0,0
3,1280,2,2.5,1,1,0,0,0,0,40.5501,...,0,0,0,0,0,0,0,0,0,0
4,783,2,1.0,1,1,1,0,0,0,38.0936,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38493,1180,2,2.0,1,1,0,1,0,0,30.6197,...,0,0,0,0,0,0,0,0,0,0
38494,1138,3,2.0,1,1,1,0,0,0,31.8045,...,0,0,1,0,0,0,0,0,0,0
38495,743,1,1.0,1,1,1,0,0,0,27.8971,...,0,0,0,0,0,0,0,0,0,0
38496,1276,3,2.0,0,0,0,0,0,0,35.0052,...,0,0,0,0,0,0,0,0,0,0


In [35]:
dftest['region_salina']=0
dftest['region_southwest TX']=0
dftest['region_st louis']=0
dftest

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,lat,...,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy,region_salina,region_southwest TX,region_st louis
0,1344,3,2.0,0,0,1,0,0,0,42.5333,...,0,0,0,0,0,0,0,0,0,0
1,1050,2,1.0,0,0,1,0,0,0,43.9393,...,0,0,1,0,0,0,0,0,0,0
2,1150,2,2.0,1,1,1,1,0,0,42.2492,...,0,0,0,0,0,0,0,0,0,0
3,1280,2,2.5,1,1,0,0,0,0,40.5501,...,0,0,0,0,0,0,0,0,0,0
4,783,2,1.0,1,1,1,0,0,0,38.0936,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38493,1180,2,2.0,1,1,0,1,0,0,30.6197,...,0,0,0,0,0,0,0,0,0,0
38494,1138,3,2.0,1,1,1,0,0,0,31.8045,...,0,0,0,0,0,0,0,0,0,0
38495,743,1,1.0,1,1,1,0,0,0,27.8971,...,0,0,0,0,0,0,0,0,0,0
38496,1276,3,2.0,0,0,0,0,0,0,35.0052,...,0,0,0,0,0,0,0,0,0,0


In [38]:
dftest['region_kansas city']=0
dftest['region_western IL']=0
dftest['type_assisted living']=0
dftest['type_land']=0
dftest

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,lat,...,state_wi,state_wv,state_wy,region_salina,region_southwest TX,region_st louis,region_kansas city,region_western IL,type_assisted living,type_land
0,1344,3,2.0,0,0,1,0,0,0,42.5333,...,0,0,0,0,0,0,0,0,0,0
1,1050,2,1.0,0,0,1,0,0,0,43.9393,...,0,0,0,0,0,0,0,0,0,0
2,1150,2,2.0,1,1,1,1,0,0,42.2492,...,0,0,0,0,0,0,0,0,0,0
3,1280,2,2.5,1,1,0,0,0,0,40.5501,...,0,0,0,0,0,0,0,0,0,0
4,783,2,1.0,1,1,1,0,0,0,38.0936,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38493,1180,2,2.0,1,1,0,1,0,0,30.6197,...,0,0,0,0,0,0,0,0,0,0
38494,1138,3,2.0,1,1,1,0,0,0,31.8045,...,0,0,0,0,0,0,0,0,0,0
38495,743,1,1.0,1,1,1,0,0,0,27.8971,...,0,0,0,0,0,0,0,0,0,0
38496,1276,3,2.0,0,0,0,0,0,0,35.0052,...,0,0,0,0,0,0,0,0,0,0


In [44]:
dftest['region_fort smith, AR']=0
dftest

,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,lat,...,state_wv,state_wy,region_salina,region_southwest TX,region_st louis,region_kansas city,region_western IL,type_assisted living,type_land,"region_fort smith, AR"
0,1344,3,2.0,0,0,1,0,0,0,42.5333,...,0,0,0,0,0,0,0,0,0,0
1,1050,2,1.0,0,0,1,0,0,0,43.9393,...,0,0,0,0,0,0,0,0,0,0
2,1150,2,2.0,1,1,1,1,0,0,42.2492,...,0,0,0,0,0,0,0,0,0,0
3,1280,2,2.5,1,1,0,0,0,0,40.5501,...,0,0,0,0,0,0,0,0,0,0
4,783,2,1.0,1,1,1,0,0,0,38.0936,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38493,1180,2,2.0,1,1,0,1,0,0,30.6197,...,0,0,0,0,0,0,0,0,0,0
38494,1138,3,2.0,1,1,1,0,0,0,31.8045,...,0,0,0,0,0,0,0,0,0,0
38495,743,1,1.0,1,1,1,0,0,0,27.8971,...,0,0,0,0,0,0,0,0,0,0
38496,1276,3,2.0,0,0,0,0,0,0,35.0052,...,0,0,0,0,0,0,0,0,0,0


In [45]:
y=dftest

In [46]:
pred = modelo.predict(y)

ValueError: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.


A falta de tiempo para la presentacion no pude resolver este inconveniente pero para obtener la predicion mediante el modelo de Random Forest es necesario que las columnas esten en el mismo orden que en el entrenamiento y al no presentar algunos items en el dataset de testeo que si estan en el entrenamiento no coinciden cuando se realizo las variable dummies. Se logro ingresar las columnas faltantes mas no ubicarlas en el mismo orden.